Importing Required Libraries

In [ ]:
import pandas as pd
import nltk
import re

Loading dataset

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Ibm/spam.csv',encoding="ISO-8859-1")

**Data Preprocessing**

Encoding the categorical column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
data['v1']=le.fit_transform(data['v1'])

In [ ]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,1,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,0,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,0,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,1,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,1,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


**bold text**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
pe=PorterStemmer()
arr=[]

In [ ]:
data.shape

(5572, 5)

In [ ]:
for i in range(5572):
  msg=data['v2'][i]
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split()
  msg=[pe.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  arr.append(msg)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=8000)

In [ ]:
x=cv.fit_transform(arr).toarray()

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
x.shape

(5572, 6221)

In [ ]:
y=data['v1'].values

In [ ]:
y

array([0, 0, 1, ..., 0, 0, 0])

Train and Test Split


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model=Sequential()

In [ ]:
model.add(Dense(units=6221,activation='relu'))
model.add(Dense(units=8000,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
140/140 [==============================] - 102s 711ms/step - loss: 0.1078 - accuracy: 0.9699
Epoch 2/5
140/140 [==============================] - 101s 722ms/step - loss: 0.0072 - accuracy: 0.9982
Epoch 3/5
140/140 [==============================] - 97s 691ms/step - loss: 0.0019 - accuracy: 0.9993
Epoch 4/5
140/140 [==============================] - 100s 712ms/step - loss: 1.9761e-04 - accuracy: 1.0000
Epoch 5/5
140/140 [==============================] - 99s 709ms/step - loss: 5.9483e-05 - accuracy: 1.0000


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/Ibm

/content/drive/MyDrive/Ibm


In [ ]:
model.save('spam_detector.h5')

Testing Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model=load_model('/content/drive/MyDrive/Ibm/spam_detector.h5')

In [ ]:
def preprocess(text):
  text=re.sub('[^a-zA-Z]',' ',text)
  text=text.lower()
  text=text.split()
  text=[pe.stem(word) for word in text if not word in set(stopwords.words('english'))]
  text=' '.join(text)
  return text

In [56]:
sms='WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

In [57]:
sms=preprocess(sms)

In [58]:
sms

'winner valu network custom select receivea prize reward claim call claim code kl valid hour'

In [59]:
pred=model.predict(cv.transform([sms]))

1/1 [==============================] - 0s 37ms/step


In [60]:
pred

array([[1.]], dtype=float32)

In [62]:
if(pred>0.5):
  print('spam msg!!!')
else:
  print('not spam')

spam msg!!!
